In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ast
import itertools
import networkx as nx
import matplotlib.font_manager as fm
import plotly.express as px

In [2]:
data = pd.read_csv(r'C:\Users\user\Downloads\5.2_dataset_forviz_add_gener.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        499 non-null    int64 
 1   순위                499 non-null    int64 
 2   영화명               499 non-null    object
 3   개봉일               499 non-null    object
 4   매출액               488 non-null    object
 5   관객수               499 non-null    object
 6   스크린수              499 non-null    object
 7   배급사               493 non-null    object
 8   coach             498 non-null    object
 9   actor_main_name   498 non-null    object
 10  actor_sub_name    483 non-null    object
 11  actor_cameo_name  277 non-null    object
 12  maker             498 non-null    object
 13  only_gener        499 non-null    object
dtypes: int64(2), object(12)
memory usage: 54.7+ KB


## 전체 데이터 기준

In [3]:
data = pd.read_csv('5.2_dataset_forviz_add_gener.csv')

# 매출액과 관객수 데이터 형태 처리
data['매출액'] = data['매출액'].str.replace(',', '').astype(float)
data['관객수'] = data['관객수'].str.replace(',', '').astype(int)

# 데이터 형태 확인 및 처리
def safe_eval(x):
    if isinstance(x, str):
        try:
            return eval(x)
        except:
            return []  # 오류 발생시 빈 리스트 반환
    return x if isinstance(x, list) else []

data['actor_main_name'] = data['actor_main_name'].apply(safe_eval)
data['actor_sub_name'] = data['actor_sub_name'].apply(safe_eval)

# 숫자를 한글식 표현으로 변환하는 함수
def korean_style_number(num, is_revenue=False):
    units = ['만', '억', '조', '경']
    steps = [10000, 100000000, 1000000000000, 10000000000000000]
    result = ''
    for i, step in enumerate(steps[::-1]):
        if num >= step:
            result += str(int(num // step)) + units[::-1][i]
            num %= step
        if is_revenue and units[::-1][i] == '억':
            break
    return result

# 장르명 정규화 함수
def normalize_genre(genre):
    genre = genre.strip()  # 공백 제거
    genre = genre.replace(' ', '')  # 중간 공백 제거
    genre = genre.lower()  # 소문자로 통일
    return genre

# 장르별로 배우 출연 횟수 및 매출, 관객수 집계
genre_actor_stats = {}

for index, row in data.iterrows():
    genres = [normalize_genre(gen) for gen in row['only_gener'].split(',')]
    actors = set(row['actor_main_name'] + row['actor_sub_name'])  # 주연과 조연 합치기
    revenue = row['매출액']
    audience = row['관객수']
    
    for genre in genres:
        if genre not in genre_actor_stats:
            genre_actor_stats[genre] = {}
        
        for actor in actors:
            if actor not in genre_actor_stats[genre]:
                genre_actor_stats[genre][actor] = {'Count': 0, 'Total Revenue': 0, 'Total Audience': 0}
            genre_actor_stats[genre][actor]['Count'] += 1
            genre_actor_stats[genre][actor]['Total Revenue'] += revenue
            genre_actor_stats[genre][actor]['Total Audience'] += audience

# 데이터 프레임으로 변환 및 한글식 숫자 변환
rows = []
for genre, actor_stats in genre_actor_stats.items():
    for actor, stats in actor_stats.items():
        if stats['Count'] >= 3:  # 3회 이상 출연한 배우만 포함
            rows.append({
                '장르': genre, 
                '배우': actor, 
                '출연 횟수': stats['Count'],
                '총 매출액': korean_style_number(stats['Total Revenue'], is_revenue=True),
                '총 관객수': korean_style_number(stats['Total Audience'])
            })

df_actor_stats = pd.DataFrame(rows)
df_actor_stats.sort_values(by=['장르', '출연 횟수', '총 매출액', '총 관객수'], ascending=[True, False, False, False], inplace=True)

# 시각화
fig = px.bar(df_actor_stats, x='장르', y='출연 횟수', color='배우', title='장르별 배우 출연 횟수 (3회 이상 출연한 배우)',
             hover_data=['총 매출액', '총 관객수'],
             category_orders={"장르": sorted(df_actor_stats['장르'].unique(), reverse=True)})
fig.update_layout(xaxis={'categoryorder':'total descending'}, showlegend=False)
fig.show()

## 2000년대

In [2]:
data = pd.read_csv('5.2_dataset_forviz_add_gener.csv')

# 매출액과 관객수 데이터 형태 처리
data['매출액'] = data['매출액'].str.replace(',', '').astype(float)
data['관객수'] = data['관객수'].str.replace(',', '').astype(int)

# 개봉일에서 연도 추출
data['year'] = pd.to_datetime(data['개봉일']).dt.year

# 1990년대 영화만 필터링
data = data[(data['year'] >= 2000) & (data['year'] < 2010)]


# 데이터 형태 확인 및 처리
def safe_eval(x):
    if isinstance(x, str):
        try:
            return eval(x)
        except:
            return []  # 오류 발생시 빈 리스트 반환
    return x if isinstance(x, list) else []

data['actor_main_name'] = data['actor_main_name'].apply(safe_eval)
data['actor_sub_name'] = data['actor_sub_name'].apply(safe_eval)

# 숫자를 한글식 표현으로 변환하는 함수
def korean_style_number(num, is_revenue=False):
    units = ['만', '억', '조', '경']
    steps = [10000, 100000000, 1000000000000, 10000000000000000]
    result = ''
    for i, step in enumerate(steps[::-1]):
        if num >= step:
            result += str(int(num // step)) + units[::-1][i]
            num %= step
        if is_revenue and units[::-1][i] == '억':
            break
    return result

# 장르명 정규화 함수
def normalize_genre(genre):
    genre = genre.strip()  # 공백 제거
    genre = genre.replace(' ', '')  # 중간 공백 제거
    genre = genre.lower()  # 소문자로 통일
    return genre

# 장르별로 배우 출연 횟수 및 매출, 관객수 집계
genre_actor_stats = {}

for index, row in data.iterrows():
    genres = [normalize_genre(gen) for gen in row['only_gener'].split(',')]
    actors = set(row['actor_main_name'] + row['actor_sub_name'])  # 주연과 조연 합치기
    revenue = row['매출액']
    audience = row['관객수']
    
    for genre in genres:
        if genre not in genre_actor_stats:
            genre_actor_stats[genre] = {}
        
        for actor in actors:
            if actor not in genre_actor_stats[genre]:
                genre_actor_stats[genre][actor] = {'Count': 0, 'Total Revenue': 0, 'Total Audience': 0}
            genre_actor_stats[genre][actor]['Count'] += 1
            genre_actor_stats[genre][actor]['Total Revenue'] += revenue
            genre_actor_stats[genre][actor]['Total Audience'] += audience

# 데이터 프레임으로 변환 및 한글식 숫자 변환
rows = []
for genre, actor_stats in genre_actor_stats.items():
    for actor, stats in actor_stats.items():
        if stats['Count'] >= 3:  # 3회 이상 출연한 배우만 포함
            rows.append({
                '장르': genre, 
                '배우': actor, 
                '출연 횟수': stats['Count'],
                '총 매출액': korean_style_number(stats['Total Revenue'], is_revenue=True),
                '총 관객수': korean_style_number(stats['Total Audience'])
            })

df_actor_stats = pd.DataFrame(rows)
df_actor_stats.sort_values(by=['장르', '출연 횟수', '총 매출액', '총 관객수'], ascending=[True, False, False, False], inplace=True)

# 시각화
fig = px.bar(df_actor_stats, x='장르', y='출연 횟수', color='배우', title='2000년대 장르별 배우 출연 횟수 (3회 이상 출연한 배우)',
             hover_data=['총 매출액', '총 관객수'],
             category_orders={"장르": sorted(df_actor_stats['장르'].unique(), reverse=True)})
fig.update_layout(xaxis={'categoryorder':'total descending'}, showlegend=False)
fig.show()
fig.write_html('jh_actor_genre_2000.html')

## 2010년대

In [3]:
data = pd.read_csv('5.2_dataset_forviz_add_gener.csv')

# 매출액과 관객수 데이터 형태 처리
data['매출액'] = data['매출액'].str.replace(',', '').astype(float)
data['관객수'] = data['관객수'].str.replace(',', '').astype(int)

# 개봉일에서 연도 추출
data['year'] = pd.to_datetime(data['개봉일']).dt.year

# 1990년대 영화만 필터링
data = data[(data['year'] >= 2010) & (data['year'] < 2020)]


# 데이터 형태 확인 및 처리
def safe_eval(x):
    if isinstance(x, str):
        try:
            return eval(x)
        except:
            return []  # 오류 발생시 빈 리스트 반환
    return x if isinstance(x, list) else []

data['actor_main_name'] = data['actor_main_name'].apply(safe_eval)
data['actor_sub_name'] = data['actor_sub_name'].apply(safe_eval)

# 숫자를 한글식 표현으로 변환하는 함수
def korean_style_number(num, is_revenue=False):
    units = ['만', '억', '조', '경']
    steps = [10000, 100000000, 1000000000000, 10000000000000000]
    result = ''
    for i, step in enumerate(steps[::-1]):
        if num >= step:
            result += str(int(num // step)) + units[::-1][i]
            num %= step
        if is_revenue and units[::-1][i] == '억':
            break
    return result

# 장르명 정규화 함수
def normalize_genre(genre):
    genre = genre.strip()  # 공백 제거
    genre = genre.replace(' ', '')  # 중간 공백 제거
    genre = genre.lower()  # 소문자로 통일
    return genre

# 장르별로 배우 출연 횟수 및 매출, 관객수 집계
genre_actor_stats = {}

for index, row in data.iterrows():
    genres = [normalize_genre(gen) for gen in row['only_gener'].split(',')]
    actors = set(row['actor_main_name'] + row['actor_sub_name'])  # 주연과 조연 합치기
    revenue = row['매출액']
    audience = row['관객수']
    
    for genre in genres:
        if genre not in genre_actor_stats:
            genre_actor_stats[genre] = {}
        
        for actor in actors:
            if actor not in genre_actor_stats[genre]:
                genre_actor_stats[genre][actor] = {'Count': 0, 'Total Revenue': 0, 'Total Audience': 0}
            genre_actor_stats[genre][actor]['Count'] += 1
            genre_actor_stats[genre][actor]['Total Revenue'] += revenue
            genre_actor_stats[genre][actor]['Total Audience'] += audience

# 데이터 프레임으로 변환 및 한글식 숫자 변환
rows = []
for genre, actor_stats in genre_actor_stats.items():
    for actor, stats in actor_stats.items():
        if stats['Count'] >= 3:  # 3회 이상 출연한 배우만 포함
            rows.append({
                '장르': genre, 
                '배우': actor, 
                '출연 횟수': stats['Count'],
                '총 매출액': korean_style_number(stats['Total Revenue'], is_revenue=True),
                '총 관객수': korean_style_number(stats['Total Audience'])
            })

df_actor_stats = pd.DataFrame(rows)
df_actor_stats.sort_values(by=['장르', '출연 횟수', '총 매출액', '총 관객수'], ascending=[True, False, False, False], inplace=True)

# 시각화
fig = px.bar(df_actor_stats, x='장르', y='출연 횟수', color='배우', title='2010년대 장르별 배우 출연 횟수 (3회 이상 출연한 배우)',
             hover_data=['총 매출액', '총 관객수'],
             category_orders={"장르": sorted(df_actor_stats['장르'].unique(), reverse=True)})
fig.update_layout(xaxis={'categoryorder':'total descending'}, showlegend=False)
fig.show()
fig.write_html('jh_actor_genre_2010.html')

## 2020년대 (배우수가 적여 2회 이상 출연으로)

In [4]:
data = pd.read_csv('5.2_dataset_forviz_add_gener.csv')

# 매출액과 관객수 데이터 형태 처리
data['매출액'] = data['매출액'].str.replace(',', '').astype(float)
data['관객수'] = data['관객수'].str.replace(',', '').astype(int)

# 개봉일에서 연도 추출
data['year'] = pd.to_datetime(data['개봉일']).dt.year

# 1990년대 영화만 필터링
data = data[(data['year'] >= 2020) ]


# 데이터 형태 확인 및 처리
def safe_eval(x):
    if isinstance(x, str):
        try:
            return eval(x)
        except:
            return []  # 오류 발생시 빈 리스트 반환
    return x if isinstance(x, list) else []

data['actor_main_name'] = data['actor_main_name'].apply(safe_eval)
data['actor_sub_name'] = data['actor_sub_name'].apply(safe_eval)

# 숫자를 한글식 표현으로 변환하는 함수
def korean_style_number(num, is_revenue=False):
    units = ['만', '억', '조', '경']
    steps = [10000, 100000000, 1000000000000, 10000000000000000]
    result = ''
    for i, step in enumerate(steps[::-1]):
        if num >= step:
            result += str(int(num // step)) + units[::-1][i]
            num %= step
        if is_revenue and units[::-1][i] == '억':
            break
    return result

# 장르명 정규화 함수
def normalize_genre(genre):
    genre = genre.strip()  # 공백 제거
    genre = genre.replace(' ', '')  # 중간 공백 제거
    genre = genre.lower()  # 소문자로 통일
    return genre

# 장르별로 배우 출연 횟수 및 매출, 관객수 집계
genre_actor_stats = {}

for index, row in data.iterrows():
    genres = [normalize_genre(gen) for gen in row['only_gener'].split(',')]
    actors = set(row['actor_main_name'] + row['actor_sub_name'])  # 주연과 조연 합치기
    revenue = row['매출액']
    audience = row['관객수']
    
    for genre in genres:
        if genre not in genre_actor_stats:
            genre_actor_stats[genre] = {}
        
        for actor in actors:
            if actor not in genre_actor_stats[genre]:
                genre_actor_stats[genre][actor] = {'Count': 0, 'Total Revenue': 0, 'Total Audience': 0}
            genre_actor_stats[genre][actor]['Count'] += 1
            genre_actor_stats[genre][actor]['Total Revenue'] += revenue
            genre_actor_stats[genre][actor]['Total Audience'] += audience

# 데이터 프레임으로 변환 및 한글식 숫자 변환
rows = []
for genre, actor_stats in genre_actor_stats.items():
    for actor, stats in actor_stats.items():
        if stats['Count'] >= 2:  # 3회 이상 출연한 배우만 포함
            rows.append({
                '장르': genre, 
                '배우': actor, 
                '출연 횟수': stats['Count'],
                '총 매출액': korean_style_number(stats['Total Revenue'], is_revenue=True),
                '총 관객수': korean_style_number(stats['Total Audience'])
            })

df_actor_stats = pd.DataFrame(rows)
df_actor_stats.sort_values(by=['장르', '출연 횟수', '총 매출액', '총 관객수'], ascending=[True, False, False, False], inplace=True)

# 시각화
fig = px.bar(df_actor_stats, x='장르', y='출연 횟수', color='배우', title='2020년대 장르별 배우 출연 횟수 (2회 이상 출연한 배우)',
             hover_data=['총 매출액', '총 관객수'],
             category_orders={"장르": sorted(df_actor_stats['장르'].unique(), reverse=True)})
fig.update_layout(xaxis={'categoryorder':'total descending'}, showlegend=False)
fig.show()
fig.write_html('jh_actor_genre_2020.html')